In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install transformers
!pip install umap-learn
!pip install transformers[torch]
import matplotlib.pyplot as plt

In [2]:
import torch

In [3]:
from transformers import AutoModelForSequenceClassification
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 6
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset
emotions = load_dataset("emotion")
from datasets import ClassLabel
emotions['train'].features['label'] = ClassLabel(
    num_classes=6,
    names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])
# import pandas as pd
# emotions.set_format(type="pandas")
# df = emotions["train"][:]
# df.head()
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error")

In [9]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.894000,0.361705,0.890000,0.887572
2,0.271300,0.235217,0.919500,0.919410


In [10]:
preds_output = trainer.predict(emotions_encoded["validation"])

In [11]:
preds_output.metrics

{'test_loss': 0.23521697521209717,
 'test_accuracy': 0.9195,
 'test_f1': 0.9194097407142078,
 'test_runtime': 3.914,
 'test_samples_per_second': 510.986,
 'test_steps_per_second': 8.176}

In [13]:
import numpy as np

In [14]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [16]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

In [17]:
# plot_confusion_matrix(y_preds, y_valid, labels)

NameError: ignored